In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as snsbb
from scipy.stats import t
from scipy.stats import norm
from linearmodels import PanelOLS
from linearmodels.panel import compare
import geopandas as gpd



In [2]:
pre_ts_analysis = pd.read_csv('Research_Firearm/pre_ts_analysis_clean.csv')

City_level_analysis =  pd.read_csv('Research_Firearm/City_lv_data.csv')

In [3]:
#*Base year is chained; 1982-1984 = 100. This table of CPI data is based upon a 1982 base of 100. What does this mean? A CPI of 195.3, as an example from 2005, indicates 95.3% inflation since 1982.

In [4]:
# CPI data (you can load it from a file if needed)
cpi_data = {
    'Year': [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'Annual': [201.6, 207.3, 215.3, 214.5, 218.1, 224.9, 229.6, 233, 236.7, 237, 240, 245.1, 251.1, 255.7, 258.8, 271, 292.7, 304.7]
}

cpi_df = pd.DataFrame(cpi_data)

# Rename the Budget column to "Budget before CPI"
pre_ts_analysis.rename(columns={'Budget': 'Budget before CPI'}, inplace=True)

# Merge pre_ts_analysis with CPI data on the "Year" column
merged_df = pd.merge(pre_ts_analysis, cpi_df, on='Year')

# Calculate the new Budget column adjusted for CPI
# Assuming the base year for CPI adjustment is 2006
base_cpi = cpi_df.loc[cpi_df['Year'] == 2006, 'Annual'].values[0]
merged_df['Budget'] = merged_df['Budget before CPI'] * (base_cpi / merged_df['Annual'])

# Drop the 'Annual' column if not needed
merged_df.drop(columns=['Annual'], inplace=True)

# Display the resulting DataFrame
print(merged_df)

      Year  Precinct  Full Time Positions  Budget before CPI          Borough  \
0     2006         1                  219           11001943  MANHATTAN SOUTH   
1     2006         5                  240           12354423  MANHATTAN SOUTH   
2     2006         6                  237           10716126  MANHATTAN SOUTH   
3     2006         7                  174            7786080  MANHATTAN SOUTH   
4     2006         9                  234           10190005  MANHATTAN SOUTH   
...    ...       ...                  ...                ...              ...   
1308  2023       114                  252           20752787     QUEENS NORTH   
1309  2023       115                  289           16525730     QUEENS NORTH   
1310  2023       120                  399           29273345    STATEN ISLAND   
1311  2023       122                  249           19873478    STATEN ISLAND   
1312  2023       123                  148           13394487    STATEN ISLAND   

      MURDER & NON NEGL. MA

In [5]:
pre_ts_analysis = merged_df

In [6]:
pre_ts_analysis

,Year,Precinct,Full Time Positions,Budget before CPI,Borough,MURDER & NON NEGL. MANSLAUGHTER,RAPE,ROBBERY,FELONY ASSAULT,BURGLARY,GRAND LARCENY,GRAND LARCENY OF MOTOR VEHICLE,TOTAL SEVEN MAJOR FELONY OFFENSES,Population_Year,Budget
0,2006,1,219,11001943,MANHATTAN SOUTH,1,4,119,94,255,1462,78,2013,59431,1.100194e+07
1,2006,5,240,12354423,MANHATTAN SOUTH,2,5,132,104,152,605,40,1040,53356,1.235442e+07
2,2006,6,237,10716126,MANHATTAN SOUTH,3,6,214,123,280,1283,70,1979,61259,1.071613e+07
3,2006,7,174,7786080,MANHATTAN SOUTH,4,7,176,105,127,362,83,864,55703,7.786080e+06
4,2006,9,234,10190005,MANHATTAN SOUTH,1,14,252,165,297,775,75,1579,76639,1.019000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,2023,114,252,20752787,QUEENS NORTH,8,40,279,509,224,920,430,2410,210231,1.373076e+07
1309,2023,115,289,16525730,QUEENS NORTH,4,27,365,481,136,863,337,2213,181382,1.093399e+07
1310,2023,120,399,29273345,STATEN ISLAND,12,19,163,470,142,414,156,1376,125104,1.936825e+07
1311,2023,122,249,19873478,STATEN ISLAND,2,8,50,169,100,474,109,912,146208,1.314898e+07


In [7]:
pre_ts_analysis

,Year,Precinct,Full Time Positions,Budget before CPI,Borough,MURDER & NON NEGL. MANSLAUGHTER,RAPE,ROBBERY,FELONY ASSAULT,BURGLARY,GRAND LARCENY,GRAND LARCENY OF MOTOR VEHICLE,TOTAL SEVEN MAJOR FELONY OFFENSES,Population_Year,Budget
0,2006,1,219,11001943,MANHATTAN SOUTH,1,4,119,94,255,1462,78,2013,59431,1.100194e+07
1,2006,5,240,12354423,MANHATTAN SOUTH,2,5,132,104,152,605,40,1040,53356,1.235442e+07
2,2006,6,237,10716126,MANHATTAN SOUTH,3,6,214,123,280,1283,70,1979,61259,1.071613e+07
3,2006,7,174,7786080,MANHATTAN SOUTH,4,7,176,105,127,362,83,864,55703,7.786080e+06
4,2006,9,234,10190005,MANHATTAN SOUTH,1,14,252,165,297,775,75,1579,76639,1.019000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,2023,114,252,20752787,QUEENS NORTH,8,40,279,509,224,920,430,2410,210231,1.373076e+07
1309,2023,115,289,16525730,QUEENS NORTH,4,27,365,481,136,863,337,2213,181382,1.093399e+07
1310,2023,120,399,29273345,STATEN ISLAND,12,19,163,470,142,414,156,1376,125104,1.936825e+07
1311,2023,122,249,19873478,STATEN ISLAND,2,8,50,169,100,474,109,912,146208,1.314898e+07


In [8]:
pre_ts_analysis.columns

Index(['Year', 'Precinct', 'Full Time Positions', 'Budget before CPI',
       'Borough', 'MURDER & NON NEGL. MANSLAUGHTER', 'RAPE', 'ROBBERY',
       'FELONY ASSAULT', 'BURGLARY', 'GRAND LARCENY',
       'GRAND LARCENY OF MOTOR VEHICLE', 'TOTAL SEVEN MAJOR FELONY OFFENSES',
       'Population_Year', 'Budget'],
      dtype='object')

In [9]:
pre_ts_analysis

,Year,Precinct,Full Time Positions,Budget before CPI,Borough,MURDER & NON NEGL. MANSLAUGHTER,RAPE,ROBBERY,FELONY ASSAULT,BURGLARY,GRAND LARCENY,GRAND LARCENY OF MOTOR VEHICLE,TOTAL SEVEN MAJOR FELONY OFFENSES,Population_Year,Budget
0,2006,1,219,11001943,MANHATTAN SOUTH,1,4,119,94,255,1462,78,2013,59431,1.100194e+07
1,2006,5,240,12354423,MANHATTAN SOUTH,2,5,132,104,152,605,40,1040,53356,1.235442e+07
2,2006,6,237,10716126,MANHATTAN SOUTH,3,6,214,123,280,1283,70,1979,61259,1.071613e+07
3,2006,7,174,7786080,MANHATTAN SOUTH,4,7,176,105,127,362,83,864,55703,7.786080e+06
4,2006,9,234,10190005,MANHATTAN SOUTH,1,14,252,165,297,775,75,1579,76639,1.019000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,2023,114,252,20752787,QUEENS NORTH,8,40,279,509,224,920,430,2410,210231,1.373076e+07
1309,2023,115,289,16525730,QUEENS NORTH,4,27,365,481,136,863,337,2213,181382,1.093399e+07
1310,2023,120,399,29273345,STATEN ISLAND,12,19,163,470,142,414,156,1376,125104,1.936825e+07
1311,2023,122,249,19873478,STATEN ISLAND,2,8,50,169,100,474,109,912,146208,1.314898e+07


## Define Violent and Crime, also calculate the percentage based crime and budget allocation on precinct level

In [10]:
# Create 'Violent Crime' column
pre_ts_analysis['Violent Crime'] = pre_ts_analysis['FELONY ASSAULT'] + pre_ts_analysis['MURDER & NON NEGL. MANSLAUGHTER'] + pre_ts_analysis['RAPE'] + pre_ts_analysis['ROBBERY']
# Create 'Property Crime' column
pre_ts_analysis['Property Crime'] = pre_ts_analysis['GRAND LARCENY'] + pre_ts_analysis['GRAND LARCENY OF MOTOR VEHICLE'] + pre_ts_analysis['BURGLARY']

In [11]:
# Calculate yearly totals
yearly_totals = pre_ts_analysis.groupby('Year').sum().reset_index()
# List of metrics for which want to calculate yearly totals
metrics = ['Violent Crime', 'Property Crime', 'Full Time Positions']
# Rename columns to *_by_year
yearly_totals = yearly_totals.rename(columns={metric: f"{metric.lower()}_by_year" for metric in metrics})
# Merge yearly totals with merged_data
merged_data = pd.merge(pre_ts_analysis, yearly_totals[['Year'] + [f"{metric.lower()}_by_year" for metric in metrics]], 
                       on='Year', how='left')

# List of metrics for which to calculate yearly totals and per capita rates
metrics = ['Violent Crime', 'Property Crime','Full Time Positions']
# Loop through each metric
for metric in metrics:
    # Compute the per capita rate
    per_capita_col_name = f"{metric}_per_capita"
    merged_data[per_capita_col_name] = merged_data[metric] / merged_data['Population_Year']    
    # Compute the percentage-based rate
    pct_col_name = f"{metric}_pct"
    yearly_col_name = f"{metric.lower()}_by_year"
    merged_data[pct_col_name] = merged_data[metric] / merged_data[yearly_col_name]

In [12]:
pre_ts_analysis = merged_data

In [13]:
# Calculate yearly totals
yearly_totals = pre_ts_analysis.groupby('Year').sum().reset_index()
# List of metrics for which want to calculate yearly totals
metrics = ['MURDER & NON NEGL. MANSLAUGHTER', 'RAPE', 'ROBBERY', 'FELONY ASSAULT', 
    'BURGLARY', 'GRAND LARCENY', 'GRAND LARCENY OF MOTOR VEHICLE', 
    'TOTAL SEVEN MAJOR FELONY OFFENSES', 'Budget']
# Rename columns to *_by_year
yearly_totals = yearly_totals.rename(columns={metric: f"{metric.lower()}_by_year" for metric in metrics})
# Merge yearly totals with merged_data
merged_data = pd.merge(pre_ts_analysis, yearly_totals[['Year'] + [f"{metric.lower()}_by_year" for metric in metrics]], 
                       on='Year', how='left')

# List of metrics for which to calculate yearly totals and per capita rates
metrics = ['MURDER & NON NEGL. MANSLAUGHTER', 'RAPE', 'ROBBERY', 'FELONY ASSAULT', 
    'BURGLARY', 'GRAND LARCENY', 'GRAND LARCENY OF MOTOR VEHICLE', 
    'TOTAL SEVEN MAJOR FELONY OFFENSES', 'Budget']
# Loop through each metric
for metric in metrics:
    # Compute the per capita rate
    per_capita_col_name = f"{metric}_per_capita"
    merged_data[per_capita_col_name] = merged_data[metric] / merged_data['Population_Year']    
    # Compute the percentage-based rate
    pct_col_name = f"{metric}_pct"
    yearly_col_name = f"{metric.lower()}_by_year"
    merged_data[pct_col_name] = merged_data[metric] / merged_data[yearly_col_name]

In [14]:
pre_ts_analysis = merged_data

In [15]:
pre_ts_analysis

,Year,Precinct,Full Time Positions,Budget before CPI,Borough,MURDER & NON NEGL. MANSLAUGHTER,RAPE,ROBBERY,FELONY ASSAULT,BURGLARY,...,BURGLARY_per_capita,BURGLARY_pct,GRAND LARCENY_per_capita,GRAND LARCENY_pct,GRAND LARCENY OF MOTOR VEHICLE_per_capita,GRAND LARCENY OF MOTOR VEHICLE_pct,TOTAL SEVEN MAJOR FELONY OFFENSES_per_capita,TOTAL SEVEN MAJOR FELONY OFFENSES_pct,Budget_per_capita,Budget_pct
0,2006,1,219,11001943,MANHATTAN SOUTH,1,4,119,94,255,...,0.004291,0.011469,0.024600,0.035392,0.001312,0.004985,0.033871,0.016574,185.121284,0.014186
1,2006,5,240,12354423,MANHATTAN SOUTH,2,5,132,104,152,...,0.002849,0.006836,0.011339,0.014646,0.000750,0.002557,0.019492,0.008563,231.547024,0.015929
2,2006,6,237,10716126,MANHATTAN SOUTH,3,6,214,123,280,...,0.004571,0.012593,0.020944,0.031059,0.001143,0.004474,0.032305,0.016294,174.931455,0.013817
3,2006,7,174,7786080,MANHATTAN SOUTH,4,7,176,105,127,...,0.002280,0.005712,0.006499,0.008763,0.001490,0.005305,0.015511,0.007114,139.778468,0.010039
4,2006,9,234,10190005,MANHATTAN SOUTH,1,14,252,165,297,...,0.003875,0.013358,0.010112,0.018761,0.000979,0.004794,0.020603,0.013000,132.961090,0.013139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,2023,114,252,20752787,QUEENS NORTH,8,40,279,509,224,...,0.001065,0.017165,0.004376,0.019979,0.002045,0.027816,0.011464,0.020210,65.312716,0.016160
1309,2023,115,289,16525730,QUEENS NORTH,4,27,365,481,136,...,0.000750,0.010421,0.004758,0.018741,0.001858,0.021800,0.012201,0.018558,60.281568,0.012868
1310,2023,120,399,29273345,STATEN ISLAND,12,19,163,470,142,...,0.001135,0.010881,0.003309,0.008991,0.001247,0.010091,0.010999,0.011539,154.817207,0.022794
1311,2023,122,249,19873478,STATEN ISLAND,2,8,50,169,100,...,0.000684,0.007663,0.003242,0.010294,0.000746,0.007051,0.006238,0.007648,89.933359,0.015475


In [16]:
pre_ts_analysis.groupby('Year')['Violent Crime_pct'].sum()

Year
2006    1.0
2007    1.0
2008    1.0
2009    1.0
2010    1.0
2011    1.0
2012    1.0
2013    1.0
2014    1.0
2015    1.0
2016    1.0
2017    1.0
2018    1.0
2019    1.0
2020    1.0
2021    1.0
2022    1.0
2023    1.0
Name: Violent Crime_pct, dtype: float64

## Create lag 1 for meature previous one year for crime and budget variable

In [17]:
# Get all columns ending with '_pct' or '_per_capita'
cols_to_lag = [col for col in pre_ts_analysis.columns if col.endswith('_pct') or col.endswith('_per_capita')]

# Loop through the columns and create a lag-1 column for each
for col in cols_to_lag:
    lag_col_name = col + '_lag1'  # Name of the new lag column
    pre_ts_analysis[lag_col_name] = pre_ts_analysis.groupby('Precinct')[col].shift(1)

In [18]:
pre_ts_analysis.columns

Index(['Year', 'Precinct', 'Full Time Positions', 'Budget before CPI',
       'Borough', 'MURDER & NON NEGL. MANSLAUGHTER', 'RAPE', 'ROBBERY',
       'FELONY ASSAULT', 'BURGLARY', 'GRAND LARCENY',
       'GRAND LARCENY OF MOTOR VEHICLE', 'TOTAL SEVEN MAJOR FELONY OFFENSES',
       'Population_Year', 'Budget', 'Violent Crime', 'Property Crime',
       'violent crime_by_year', 'property crime_by_year',
       'full time positions_by_year', 'Violent Crime_per_capita',
       'Violent Crime_pct', 'Property Crime_per_capita', 'Property Crime_pct',
       'Full Time Positions_per_capita', 'Full Time Positions_pct',
       'murder & non negl. manslaughter_by_year', 'rape_by_year',
       'robbery_by_year', 'felony assault_by_year', 'burglary_by_year',
       'grand larceny_by_year', 'grand larceny of motor vehicle_by_year',
       'total seven major felony offenses_by_year', 'budget_by_year',
       'MURDER & NON NEGL. MANSLAUGHTER_per_capita',
       'MURDER & NON NEGL. MANSLAUGHTER_pct', 'R

# Paper Analysis

In [19]:
# Creating a lagged version of the Violent and Property Crime

# Defining the regression formula
regression_formula = "Budget_per_capita ~ Q('Violent Crime_per_capita_lag1') + Q('Property Crime_per_capita_lag1') + Budget_per_capita_lag1"

# Running the regression
model = smf.ols(formula=regression_formula, data=pre_ts_analysis).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      Budget_per_capita   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     5657.
Date:                Fri, 17 May 2024   Prob (F-statistic):               0.00
Time:                        09:32:36   Log-Likelihood:                -4994.7
No. Observations:                1240   AIC:                             9997.
Df Residuals:                    1236   BIC:                         1.002e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [20]:
# Creating a lagged version of the Violent and Property Crime pct

# Defining the regression formula
regression_formula = "Budget_pct ~ Q('Violent Crime_pct_lag1') + Q('Property Crime_pct_lag1') + Budget_pct_lag1"

# Running the regression
model = smf.ols(formula=regression_formula, data=pre_ts_analysis).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             Budget_pct   R-squared:                       0.908
Model:                            OLS   Adj. R-squared:                  0.907
Method:                 Least Squares   F-statistic:                     4045.
Date:                Fri, 17 May 2024   Prob (F-statistic):               0.00
Time:                        09:32:36   Log-Likelihood:                 6793.4
No. Observations:                1240   AIC:                        -1.358e+04
Df Residuals:                    1236   BIC:                        -1.356e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [21]:
filtered_pre_ts_analysis = pre_ts_analysis[(pre_ts_analysis['Year'] >= 2007) & (pre_ts_analysis['Year'] <= 2023)]


In [22]:
# Reset display options to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [23]:

# Assuming 'filtered_pre_ts_analysis' is your DataFrame with proper columns.
X = filtered_pre_ts_analysis[['Violent Crime_pct_lag1', 'Property Crime_pct_lag1', 'Budget_pct_lag1']]
y = filtered_pre_ts_analysis['Budget_pct']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:             Budget_pct   R-squared:                       0.908
Model:                            OLS   Adj. R-squared:                  0.907
Method:                 Least Squares   F-statistic:                     4045.
Date:                Fri, 17 May 2024   Prob (F-statistic):               0.00
Time:                        09:32:36   Log-Likelihood:                -282.62
No. Observations:                1240   AIC:                             573.2
Df Residuals:                    1236   BIC:                             593.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    -

In [24]:
predictors_df_2 = filtered_pre_ts_analysis[['GRAND LARCENY_pct_lag1','BURGLARY_pct_lag1','GRAND LARCENY OF MOTOR VEHICLE_pct_lag1','FELONY ASSAULT_pct_lag1','MURDER & NON NEGL. MANSLAUGHTER_pct_lag1','RAPE_pct_lag1', 'ROBBERY_pct_lag1', 'Budget_pct_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                                   Variable        VIF
0                    GRAND LARCENY_pct_lag1   7.528083
1                         BURGLARY_pct_lag1  16.414193
2   GRAND LARCENY OF MOTOR VEHICLE_pct_lag1   8.663947
3                   FELONY ASSAULT_pct_lag1  32.980409
4  MURDER & NON NEGL. MANSLAUGHTER_pct_lag1   6.563781
5                             RAPE_pct_lag1  13.413026
6                          ROBBERY_pct_lag1  32.808150
7                           Budget_pct_lag1  15.690278


In [25]:
predictors_df_2 = filtered_pre_ts_analysis[['GRAND LARCENY_pct_lag1','BURGLARY_pct_lag1','GRAND LARCENY OF MOTOR VEHICLE_pct_lag1','MURDER & NON NEGL. MANSLAUGHTER_pct_lag1','RAPE_pct_lag1', 'ROBBERY_pct_lag1', 'Budget_pct_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                                   Variable        VIF
0                    GRAND LARCENY_pct_lag1   7.198934
1                         BURGLARY_pct_lag1  16.279178
2   GRAND LARCENY OF MOTOR VEHICLE_pct_lag1   8.663947
3  MURDER & NON NEGL. MANSLAUGHTER_pct_lag1   5.720214
4                             RAPE_pct_lag1  12.592054
5                          ROBBERY_pct_lag1  17.812288
6                           Budget_pct_lag1  14.813462


In [26]:
predictors_df_2 = filtered_pre_ts_analysis[['GRAND LARCENY_pct_lag1','BURGLARY_pct_lag1','GRAND LARCENY OF MOTOR VEHICLE_pct_lag1','MURDER & NON NEGL. MANSLAUGHTER_pct_lag1','RAPE_pct_lag1', 'Budget_pct_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                                   Variable        VIF
0                    GRAND LARCENY_pct_lag1   7.167830
1                         BURGLARY_pct_lag1  15.273549
2   GRAND LARCENY OF MOTOR VEHICLE_pct_lag1   8.619399
3  MURDER & NON NEGL. MANSLAUGHTER_pct_lag1   4.581264
4                             RAPE_pct_lag1   9.823855
5                           Budget_pct_lag1  14.768607


In [27]:
predictors_df_2 = filtered_pre_ts_analysis[['GRAND LARCENY_pct_lag1','GRAND LARCENY OF MOTOR VEHICLE_pct_lag1','MURDER & NON NEGL. MANSLAUGHTER_pct_lag1','RAPE_pct_lag1', 'Budget_pct_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                                   Variable        VIF
0                    GRAND LARCENY_pct_lag1   6.026963
1   GRAND LARCENY OF MOTOR VEHICLE_pct_lag1   5.584618
2  MURDER & NON NEGL. MANSLAUGHTER_pct_lag1   4.570835
3                             RAPE_pct_lag1   9.753610
4                           Budget_pct_lag1  14.099469


In [28]:
predictors_df_2 = filtered_pre_ts_analysis[['GRAND LARCENY_pct_lag1','GRAND LARCENY OF MOTOR VEHICLE_pct_lag1','MURDER & NON NEGL. MANSLAUGHTER_pct_lag1','RAPE_pct_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                                   Variable       VIF
0                    GRAND LARCENY_pct_lag1  3.188364
1   GRAND LARCENY OF MOTOR VEHICLE_pct_lag1  5.426184
2  MURDER & NON NEGL. MANSLAUGHTER_pct_lag1  4.180618
3                             RAPE_pct_lag1  8.294728


In [29]:
X = filtered_pre_ts_analysis[['GRAND LARCENY_pct_lag1','MURDER & NON NEGL. MANSLAUGHTER_pct_lag1','RAPE_pct_lag1']]
y = filtered_pre_ts_analysis['Budget_pct']
X = sm.add_constant(filtered_pre_ts_analysis[['GRAND LARCENY_pct_lag1','MURDER & NON NEGL. MANSLAUGHTER_pct_lag1','RAPE_pct_lag1']])

# Fit the OLS regression model
model_4 = sm.OLS(y, X).fit()
print(model_4.summary())

                            OLS Regression Results                            
Dep. Variable:             Budget_pct   R-squared:                       0.593
Model:                            OLS   Adj. R-squared:                  0.592
Method:                 Least Squares   F-statistic:                     600.4
Date:                Fri, 17 May 2024   Prob (F-statistic):          1.06e-240
Time:                        09:32:36   Log-Likelihood:                 5874.4
No. Observations:                1240   AIC:                        -1.174e+04
Df Residuals:                    1236   BIC:                        -1.172e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [30]:
X = filtered_pre_ts_analysis[['GRAND LARCENY_pct_lag1','GRAND LARCENY OF MOTOR VEHICLE_pct_lag1','MURDER & NON NEGL. MANSLAUGHTER_pct_lag1','RAPE_pct_lag1']]
y = filtered_pre_ts_analysis['Budget_pct']
X = sm.add_constant(filtered_pre_ts_analysis[['GRAND LARCENY_pct_lag1','GRAND LARCENY OF MOTOR VEHICLE_pct_lag1','MURDER & NON NEGL. MANSLAUGHTER_pct_lag1','RAPE_pct_lag1']])

# Fit the OLS regression model
model_4 = sm.OLS(y, X).fit()
print(model_4.summary())

                            OLS Regression Results                            
Dep. Variable:             Budget_pct   R-squared:                       0.594
Model:                            OLS   Adj. R-squared:                  0.593
Method:                 Least Squares   F-statistic:                     451.9
Date:                Fri, 17 May 2024   Prob (F-statistic):          5.68e-240
Time:                        09:32:36   Log-Likelihood:                 5876.0
No. Observations:                1240   AIC:                        -1.174e+04
Df Residuals:                    1235   BIC:                        -1.172e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [31]:
# Assuming 'filtered_pre_ts_analysis' is your DataFrame with proper columns.
X = filtered_pre_ts_analysis[['GRAND LARCENY_pct_lag1','GRAND LARCENY OF MOTOR VEHICLE_pct_lag1', 'MURDER & NON NEGL. MANSLAUGHTER_pct_lag1','RAPE_pct_lag1']]
y = filtered_pre_ts_analysis['Budget_pct']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:             Budget_pct   R-squared:                       0.594
Model:                            OLS   Adj. R-squared:                  0.593
Method:                 Least Squares   F-statistic:                     451.9
Date:                Fri, 17 May 2024   Prob (F-statistic):          5.68e-240
Time:                        09:32:36   Log-Likelihood:                -1199.9
No. Observations:                1240   AIC:                             2410.
Df Residuals:                    1235   BIC:                             2435.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

## Potential add _per_capita

In [33]:

# Assuming 'filtered_pre_ts_analysis' is your DataFrame with proper columns.
X = filtered_pre_ts_analysis[['Violent Crime_per_capita_lag1', 'Property Crime_per_capita_lag1', 'Budget_per_capita_lag1']]
y = filtered_pre_ts_analysis['Budget_per_capita']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:      Budget_per_capita   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     5657.
Date:                Fri, 17 May 2024   Prob (F-statistic):               0.00
Time:                        09:32:36   Log-Likelihood:                -91.262
No. Observations:                1240   AIC:                             190.5
Df Residuals:                    1236   BIC:                             211.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

In [34]:
predictors_df_2 = filtered_pre_ts_analysis[['Violent Crime_per_capita_lag1', 'Property Crime_per_capita_lag1', 'Budget_per_capita_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                         Variable        VIF
0   Violent Crime_per_capita_lag1   5.992767
1  Property Crime_per_capita_lag1   6.406695
2          Budget_per_capita_lag1  10.636206


In [35]:
# Assuming 'filtered_pre_ts_analysis' is your DataFrame with proper columns.
X = filtered_pre_ts_analysis[['Violent Crime_per_capita_lag1', 'Property Crime_per_capita_lag1']]
y = filtered_pre_ts_analysis['Budget_per_capita']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:      Budget_per_capita   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.453
Method:                 Least Squares   F-statistic:                     514.1
Date:                Fri, 17 May 2024   Prob (F-statistic):          3.12e-163
Time:                        09:32:37   Log-Likelihood:                -1383.9
No. Observations:                1240   AIC:                             2774.
Df Residuals:                    1237   BIC:                             2789.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

In [36]:
# Assuming 'filtered_pre_ts_analysis' is your DataFrame with proper columns.
X = filtered_pre_ts_analysis[['Budget_per_capita_lag1']]
y = filtered_pre_ts_analysis['Budget_per_capita']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:      Budget_per_capita   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                 1.634e+04
Date:                Fri, 17 May 2024   Prob (F-statistic):               0.00
Time:                        09:32:37   Log-Likelihood:                -114.10
No. Observations:                1240   AIC:                             232.2
Df Residuals:                    1238   BIC:                             242.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                  -5.37

In [37]:
predictors_df_2 = filtered_pre_ts_analysis[['GRAND LARCENY_per_capita_lag1','BURGLARY_per_capita_lag1','GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1','FELONY ASSAULT_per_capita_lag1','MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1','RAPE_per_capita_lag1', 'ROBBERY_per_capita_lag1', 'Budget_per_capita_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                                          Variable        VIF
0                    GRAND LARCENY_per_capita_lag1   5.889214
1                         BURGLARY_per_capita_lag1  13.487514
2   GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1   5.890423
3                   FELONY ASSAULT_per_capita_lag1  13.278736
4  MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1   4.840056
5                             RAPE_per_capita_lag1   9.250470
6                          ROBBERY_per_capita_lag1  16.351128
7                           Budget_per_capita_lag1  12.338139


In [38]:
predictors_df_2 = filtered_pre_ts_analysis[['GRAND LARCENY_per_capita_lag1','BURGLARY_per_capita_lag1','GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1','FELONY ASSAULT_per_capita_lag1','MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1','RAPE_per_capita_lag1', 'Budget_per_capita_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                                          Variable        VIF
0                    GRAND LARCENY_per_capita_lag1   5.888721
1                         BURGLARY_per_capita_lag1  11.708315
2   GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1   5.872446
3                   FELONY ASSAULT_per_capita_lag1  11.319595
4  MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1   4.373352
5                             RAPE_per_capita_lag1   9.096182
6                           Budget_per_capita_lag1  12.297980


In [39]:
predictors_df_2 = filtered_pre_ts_analysis[['GRAND LARCENY_per_capita_lag1','BURGLARY_per_capita_lag1','GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1','FELONY ASSAULT_per_capita_lag1','MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1','RAPE_per_capita_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                                          Variable        VIF
0                    GRAND LARCENY_per_capita_lag1   4.124782
1                         BURGLARY_per_capita_lag1  11.234866
2   GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1   5.787411
3                   FELONY ASSAULT_per_capita_lag1  10.276021
4  MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1   4.371026
5                             RAPE_per_capita_lag1   8.452015


In [40]:
predictors_df_2 = filtered_pre_ts_analysis[['GRAND LARCENY_per_capita_lag1','GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1','FELONY ASSAULT_per_capita_lag1','MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1','RAPE_per_capita_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                                          Variable        VIF
0                    GRAND LARCENY_per_capita_lag1   2.640496
1   GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1   3.371042
2                   FELONY ASSAULT_per_capita_lag1  10.092682
3  MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1   4.277318
4                             RAPE_per_capita_lag1   8.450958


In [41]:
predictors_df_2 = filtered_pre_ts_analysis[['GRAND LARCENY_per_capita_lag1','GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1','MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1','RAPE_per_capita_lag1']]
# Add a constant to the model (intercept)

# Calculate VIF for each predictor variable
vif = pd.DataFrame()
vif["Variable"] = predictors_df_2.columns
vif["VIF"] = [variance_inflation_factor(predictors_df_2.values, i) for i in range(predictors_df_2.shape[1])]

print(vif)

                                          Variable       VIF
0                    GRAND LARCENY_per_capita_lag1  2.548589
1   GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1  3.343485
2  MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1  3.437560
3                             RAPE_per_capita_lag1  5.039135


In [42]:
# Assuming 'filtered_pre_ts_analysis' is your DataFrame with proper columns.
X = filtered_pre_ts_analysis[['GRAND LARCENY_per_capita_lag1','GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1','MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1','RAPE_per_capita_lag1']]
y = filtered_pre_ts_analysis['Budget_per_capita']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:      Budget_per_capita   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.497
Method:                 Least Squares   F-statistic:                     307.0
Date:                Fri, 17 May 2024   Prob (F-statistic):          2.22e-183
Time:                        09:32:37   Log-Likelihood:                -1330.9
No. Observations:                1240   AIC:                             2672.
Df Residuals:                    1235   BIC:                             2698.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

In [43]:
# Assuming 'filtered_pre_ts_analysis' is your DataFrame with proper columns.
X = filtered_pre_ts_analysis[['GRAND LARCENY_per_capita_lag1','GRAND LARCENY OF MOTOR VEHICLE_per_capita_lag1','MURDER & NON NEGL. MANSLAUGHTER_per_capita_lag1','RAPE_per_capita_lag1']]
y = filtered_pre_ts_analysis['Budget_per_capita']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:      Budget_per_capita   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.497
Method:                 Least Squares   F-statistic:                     307.0
Date:                Fri, 17 May 2024   Prob (F-statistic):          2.22e-183
Time:                        09:32:37   Log-Likelihood:                -1330.9
No. Observations:                1240   AIC:                             2672.
Df Residuals:                    1235   BIC:                             2698.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

## Supplement

In [44]:
# Load the data (assuming the data is in a CSV file named 'City_level_analysis.csv')
df = City_level_analysis

# Create lagged variables
df['Budget_lag1'] = df['Budget'].shift(1)
df['TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_lag1'] = df['TOTAL SEVEN MAJOR FELONY OFFENSES'].shift(1)
df['Violent_Crime_lag1'] = df['Violent Crime'].shift(1)
df['Property_Crime_lag1'] = df['Property Crime'].shift(1)
df['Budget_per_capita'] = df['Budget'] / df['Population_Year']
df['TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_per_capita'] = df['TOTAL SEVEN MAJOR FELONY OFFENSES'] / df['Population_Year']
df['Violent_Crime_per_capita'] = df['Violent Crime'] / df['Population_Year']
df['Property_Crime_per_capita'] = df['Property Crime'] / df['Population_Year']
df['Budget_lag1_per_capita'] = df['Budget_lag1'] / df['Population_Year']
df['TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_lag1_per_capita'] = df['TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_lag1'] / df['Population_Year']
df['Violent_Crime_lag1_per_capita'] = df['Violent_Crime_lag1'] / df['Population_Year']
df['Property_Crime_lag1_per_capita'] = df['Property_Crime_lag1'] / df['Population_Year']

df = df[df['Year' ]> 2006]

In [45]:
df.columns

Index(['Year', 'Budget', 'TOTAL SEVEN MAJOR FELONY OFFENSES', 'Violent Crime',
       'Property Crime', 'Population_Year', 'Budget_lag1',
       'TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_lag1', 'Violent_Crime_lag1',
       'Property_Crime_lag1', 'Budget_per_capita',
       'TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_per_capita',
       'Violent_Crime_per_capita', 'Property_Crime_per_capita',
       'Budget_lag1_per_capita',
       'TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_lag1_per_capita',
       'Violent_Crime_lag1_per_capita', 'Property_Crime_lag1_per_capita'],
      dtype='object')

In [46]:
X = df[["TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_lag1"]]
y = df['Budget']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:                 Budget   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     9.871
Date:                Fri, 17 May 2024   Prob (F-statistic):            0.00672
Time:                        09:32:37   Log-Likelihood:                -19.309
No. Observations:                  17   AIC:                             42.62
Df Residuals:                      15   BIC:                             44.28
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

C:\Users\yinwe\Work_software\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [47]:
df

,Year,Budget,TOTAL SEVEN MAJOR FELONY OFFENSES,Violent Crime,Property Crime,Population_Year,Budget_lag1,TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_lag1,Violent_Crime_lag1,Property_Crime_lag1,Budget_per_capita,TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_per_capita,Violent_Crime_per_capita,Property_Crime_per_capita,Budget_lag1_per_capita,TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_lag1_per_capita,Violent_Crime_lag1_per_capita,Property_Crime_lag1_per_capita
1,2007,8.769038e+08,114173,40323,73850,7802093,7.755708e+08,121457.0,42268.0,79189.0,112.393407,0.014634,0.005168,0.009465,99.405478,0.015567,0.005418,0.010150
2,2008,8.690889e+08,111433,39739,71694,7861945,8.769038e+08,114173.0,40323.0,73850.0,110.543752,0.014174,0.005055,0.009119,111.537770,0.014522,0.005129,0.009393
3,2009,8.723303e+08,100851,36389,64462,7921799,8.690889e+08,111433.0,39739.0,71694.0,110.117696,0.012731,0.004594,0.008137,109.708527,0.014067,0.005016,0.009050
4,2010,8.703947e+08,99614,37662,61952,7981683,8.723303e+08,100851.0,36389.0,64462.0,109.049014,0.012480,0.004719,0.007762,109.291518,0.012635,0.004559,0.008076
5,2011,1.021553e+09,101444,39474,61970,8041507,8.703947e+08,99614.0,37662.0,61952.0,127.035048,0.012615,0.004909,0.007706,108.237755,0.012387,0.004683,0.007704
6,2012,9.349572e+08,106178,40819,65359,8101366,1.021553e+09,101444.0,39474.0,61970.0,115.407350,0.013106,0.005039,0.008068,126.096417,0.012522,0.004873,0.007649
7,2013,9.658835e+08,105549,40242,65307,8161213,9.349572e+08,106178.0,40819.0,65359.0,118.350481,0.012933,0.004931,0.008002,114.561056,0.013010,0.005002,0.008008
8,2014,9.531618e+08,101052,36980,64072,8238935,9.658835e+08,105549.0,40242.0,65307.0,115.689933,0.012265,0.004488,0.007777,117.234022,0.012811,0.004884,0.007927
9,2015,9.478612e+08,98749,37429,61320,8280937,9.531618e+08,101052.0,36980.0,64072.0,114.463038,0.011925,0.004520,0.007405,115.103138,0.012203,0.004466,0.007737
10,2016,9.474815e+08,94929,36483,58446,8340778,9.478612e+08,98749.0,37429.0,61320.0,113.596293,0.011381,0.004374,0.007007,113.641822,0.011839,0.004487,0.007352


In [48]:
X = df[["TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_lag1_per_capita"]]
y = df['Budget_per_capita']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:      Budget_per_capita   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.055
Method:                 Least Squares   F-statistic:                    0.1659
Date:                Fri, 17 May 2024   Prob (F-statistic):              0.690
Time:                        09:32:37   Log-Likelihood:                -23.513
No. Observations:                  17   AIC:                             51.03
Df Residuals:                      15   BIC:                             52.69
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

C:\Users\yinwe\Work_software\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [49]:
X = df[["TOTAL_SEVEN_MAJOR_FELONY_OFFENSES_lag1", "Budget_lag1"]]
y = df['Budget']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:                 Budget   R-squared:                       0.412
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     4.913
Date:                Fri, 17 May 2024   Prob (F-statistic):             0.0242
Time:                        09:32:37   Log-Likelihood:                -19.087
No. Observations:                  17   AIC:                             44.17
Df Residuals:                      14   BIC:                             46.67
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

C:\Users\yinwe\Work_software\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [50]:
X = City_level_analysis[["Violent Crime", "Property Crime"]]
y = City_level_analysis['Budget']

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X_standardized = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)  # Match the indices

# Standardize the dependent variable
y_mean = y.mean()
y_std = y.std()
y_standardized = (y - y_mean) / y_std
y_standardized.index = X.index  # Ensure the indices are the same

# Add a constant to the model (intercept)
X_standardized = sm.add_constant(X_standardized)  # This will now retain the column names and index

# Fit the OLS model with standardized variables
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Fit the OLS regression model
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:                 Budget   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     19.66
Date:                Fri, 17 May 2024   Prob (F-statistic):           6.44e-05
Time:                        09:32:37   Log-Likelihood:                -13.446
No. Observations:                  18   AIC:                             32.89
Df Residuals:                      15   BIC:                             35.56
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.596e-15      0.132   1.21e

C:\Users\yinwe\Work_software\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
